# Modeling an orogenic system using FastScape: 6. Response to tectonic variation

![Henry Mountains](HenryMountains.jpg "Henry Mountains")

In [ ]:
import numpy as np
import matplotlib.pyplot as plt
#plt.style.use('dark_background')

import xsimlab as xs
import xarray as xr
%load_ext xsimlab.ipython

from fastscape.models import basic_model


Finally we are going to see how the source area (the mountain) reacts to cyclic perturbation in uplift rate.

This does not make much sense as tectonics is unlikely to vary in a periodic manner. However, we will apply the *standard* method of computing the response of a system to periodic perturbation to characterize it.

You should go through this notebook step-by-step and try to understand what is being done. It is very similar to the previous notebook so this should not be too difficult. You should also try to run this notebook after changing the period of the climate cycles to see how it affects the system response (in terms of flux amplitude and lag).

In [ ]:
spl_model = basic_model.drop_processes('diffusion')

In [ ]:
m = 0.4
n = 1
neq = 101
Kf = 1e-5
U = 1e-3

nstep = 201
teq = 1e7
period = 1e8
tfinal = teq + 5*period

tim1 = np.linspace(0,teq,101)
tim2 = np.linspace(teq + period/10, tfinal, 100)
tim = np.concatenate((tim1,tim2))

tecto = 1 + 0.5*np.sin(2*np.pi*(tim-teq)/period)

U_tim = np.where(tim>teq, U*tecto, U)

U_xr = xr.DataArray(data=U_tim, dims='time')

fig, ax = plt.subplots(nrows = 1, ncols = 1, sharex=False, sharey=True, figsize=(12,7))

ax.plot(tim, U_tim)

In [ ]:
# %create_setup spl_model --default --verbose
import xsimlab as xs

ds_in = xs.create_setup(
    model=spl_model,
    clocks={'time': tim},
    input_vars={
        # nb. of grid nodes in (y, x)
        'grid__shape': [101,101],
        # total grid length in (y, x)
        'grid__length': [1e5,1e5],
        # node status at borders
        'boundary__status': ['fixed_value', 'core', 'looped', 'looped'],
        # uplift rate
        'uplift__rate': U_xr,
        # random seed
        'init_topography__seed': None,
        # bedrock channel incision coefficient
        'spl__k_coef': Kf,
        # drainage area exponent
        'spl__area_exp': m,
        # slope exponent
        'spl__slope_exp': n,
    },
    output_vars={'topography__elevation': 'time'}
)


In [ ]:
import zarr
zgroup = zarr.group("output.zarr", overwrite=True)

In [ ]:
with xs.monitoring.ProgressBar():
    ds_out = ds_in.xsimlab.run(model=spl_model, store=zgroup)

In [ ]:
from ipyfastscape import TopoViz3d

app = TopoViz3d(ds_out, canvas_height=300, time_dim="time")

app.components['background_color'].set_color('lightgray')
app.components['vertical_exaggeration'].set_factor(5)
app.components['timestepper'].go_to_time(ds_out.time[-1])

app.show()

The flux of material, $\phi$, coming out of the orogen can be calculated according to:

$$\phi=\frac{U-<\frac{\partial h}{\partial t}>}{U}$$

In [ ]:
fig, ax = plt.subplots(nrows=1, ncols=1, sharex=False, sharey=True, figsize=(12, 7))

((ds_out.uplift__rate - ds_out.topography__elevation.differentiate("time").mean(("x", "y"))) / ds_out.uplift__rate
).plot(ax=ax, label="Flux")
(ds_out.uplift__rate / U).plot(ax=ax, label="Tecto")
(ds_out.topography__elevation.mean(("x", "y")) / ds_out.topography__elevation.mean()).plot(ax=ax, label="Topo")

ax.legend()
ax.set_xlim((teq,tfinal))